In [16]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
! pip install thop

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import os
import os.path
import wget
from zipfile import ZipFile
import copy
import numpy as np
from torchvision import datasets, transforms
import torch

from utils.sampling import cifar_noniid, cifar_iid
from utils.options import args_parser
from models.Update import LocalUpdate
from models.Nets import MobileNetV2
from models.Fed import FedAvg
from models.Test import test_img
from utils.util import setup_seed
from utils.util import exp_details
from utils.dataset_reader import TinyImageNetDataset
from datetime import datetime
import torchvision.models as models
from torch.utils.tensorboard import SummaryWriter

In [32]:
class Argments:
    def __init__(self):
        self.epochs = 10
        self.num_users = 5
        self.frac = 0.5
        self.local_ep = 5
        self.local_bs = 32
        self.test_bs = 32
        self.lr = 0.01
        self.momentum = 0.5
        self.split = 'user'
        
        self.model = 'mobilenet'
        
        self.rebuild = 1
        self.struct = 1
        self.dataset = 'imagenet'
        self.iid = True
        self.alpha = 0.5
        self.num_classes = 200
        self.num_channels = 3
        self.gpu = 0
        self.stopping_rounds = 10
        self.verbose = 1
        self.debug = 1
        self.seed = 1

In [33]:
# parse args
args = Argments()
args.device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() and args.gpu != -1 else 'cpu')
setup_seed(args.seed)
exp_details(args)


---------------Experimental details:-------------

	Model           : mobilenet
	tLearning       : 0.01
	Global Rounds   : 10


----------------Federated parameters:------------

	IID
	Fraction of users  : 0.5
	Local Batch size   : 32
	Local Epochs       : 5



In [34]:
current_time = datetime.now().strftime('%b.%d_%H.%M.%S')
TAG = 'exp/fed/{}_{}_{}_C{}_iid{}_{}_user{}_{}'.format(args.dataset, args.model, args.epochs, args.frac, args.iid,
                                                        args.alpha, args.num_users, current_time)
# TAG = f'alpha_{alpha}/data_distribution'
logdir = f'runs/{TAG}' if not args.debug else f'runs2/{TAG}'
writer = SummaryWriter(logdir)

In [35]:
if args.dataset == 'imagenet':
    TINY_IMAGENET_ROOT = 'data/tiny-imagenet-200/'
    if os.path.exists('tiny-imagenet-200.zip') == False:
        print('\nDownload dataset\n')
        url = 'http://cs231n.stanford.edu/tiny-imagenet-200.zip'
        tiny_imgdataset = wget.download(url, out = os.getcwd())
        with ZipFile('tiny-imagenet-200.zip', 'r') as zip_ref:
            zip_ref.extractall('data/')
    else:
        print('\nDataset is already downloaded\n')


    dataset_train = datasets.ImageFolder(
        os.path.join('data/', 'tiny-imagenet-200', 'train'),
        transform=transforms.Compose(
            [
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.RandomRotation(20),
                transforms.RandomHorizontalFlip(0.5),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ]
        )
    )
    dataset_test = TinyImageNetDataset(
        img_path=os.path.join('data/', 'tiny-imagenet-200', 'val', 'images'), 
        gt_path=os.path.join('data/', 'tiny-imagenet-200', 'val', 'val_annotations.txt'),
        class_to_idx=dataset_train.class_to_idx.copy(),
        transform=transforms.Compose(
            [
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ]
        )
    )
    




Dataset is already downloaded



In [36]:
if args.iid:
    print('start separate dataset for iid')
    dict_users = cifar_iid(dataset_train, args.num_users)
    print('end')
else:
    print('start separate dataset for non-iid')
    dict_users, _ = cifar_noniid(dataset_train, args.num_users, args.alpha)
    for k, v in dict_users.items():
        writer.add_histogram(f'user_{k}/data_distribution',
                            np.array(dataset_train.targets)[v],
                            bins=np.arange(11))
        writer.add_histogram(f'all_user/data_distribution',
                            np.array(dataset_train.targets)[v],
                            bins=np.arange(11), global_step=k)
    print('end')
    

start separate dataset for iid
end


In [37]:
# build model
if args.model == 'mobilenet' and args.dataset == 'imagenet':
    # net_glob = MobileNetV2().to(args.device)
    net_glob = models.mobilenet_v3_small(pretrained=True).to(args.device)
else:
    exit('Error: unrecognized model')
print(net_glob)
net_glob.train()

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [38]:
# copy weights
w_glob = net_glob.state_dict()

In [39]:
# training
loss_train = []
cv_loss, cv_acc = [], []
val_loss_pre, counter = 0, 0
net_best = None
best_loss = None
val_acc_list, net_list = [], []
test_best_acc = 0.0

In [40]:
for iter in range(args.epochs):
    print(f'\nGlobal epoch {iter}\n')
    w_locals, loss_locals = [], []
    m = max(int(args.frac * args.num_users), 1)
    idxs_users = np.random.choice(range(args.num_users), m, replace=False)
    for idx in idxs_users:
        print(f'\nclient {idx}\n')
        local = LocalUpdate(args=args, dataset=dataset_train, idxs=dict_users[idx])
        w, loss = local.train(net=copy.deepcopy(net_glob).to(args.device))
        w_locals.append(w)
        loss_locals.append(loss)
    # update global weights
    w_glob = FedAvg(w_locals)

    # copy weight to net_glob
    net_glob.load_state_dict(w_glob)

    # print loss
    loss_avg = sum(loss_locals) / len(loss_locals)
    print('==============================')
    print('Round {:3d}, Train loss {:.3f}'.format(iter, loss_avg))
    loss_train.append(loss_avg)
    writer.add_scalar('train_loss', loss_avg, iter)
    test_acc, test_loss = test_img(net_glob, dataset_test, args)
    writer.add_scalar('test_loss', test_loss, iter)
    writer.add_scalar('test_acc', test_acc, iter)
    print('==============================')
    save_info = {
        "model": net_glob.state_dict(),
        "epoch": iter
    }
    # save model weights
    if (iter+1) % 500 == 0:
        save_path = f'./save2/{TAG}_{iter+1}es' if args.debug else f'./save/{TAG}_{iter+1}es'
        torch.save(save_info, save_path)
    if iter > 100 and test_acc > test_best_acc:
        test_best_acc = test_acc
        save_path = f'./save2/{TAG}_bst' if args.debug else f'./save/{TAG}_bst'
        torch.save(save_info, save_path)


Global epoch 0


client 4

Update Epoch: 0 [0/20000 (0%)]	Loss: 9.824431
Update Epoch: 0 [320/20000 (2%)]	Loss: 7.755905
Update Epoch: 0 [640/20000 (3%)]	Loss: 7.338758
Update Epoch: 0 [960/20000 (5%)]	Loss: 6.574852
Update Epoch: 0 [1280/20000 (6%)]	Loss: 6.347293
Update Epoch: 0 [1600/20000 (8%)]	Loss: 6.323892
Update Epoch: 0 [1920/20000 (10%)]	Loss: 5.944541
Update Epoch: 0 [2240/20000 (11%)]	Loss: 5.772562
Update Epoch: 0 [2560/20000 (13%)]	Loss: 6.269417
Update Epoch: 0 [2880/20000 (14%)]	Loss: 5.795028
Update Epoch: 0 [3200/20000 (16%)]	Loss: 5.612286
Update Epoch: 0 [3520/20000 (18%)]	Loss: 5.409889
Update Epoch: 0 [3840/20000 (19%)]	Loss: 6.174156
Update Epoch: 0 [4160/20000 (21%)]	Loss: 5.764199
Update Epoch: 0 [4480/20000 (22%)]	Loss: 5.522725
Update Epoch: 0 [4800/20000 (24%)]	Loss: 5.961246
Update Epoch: 0 [5120/20000 (26%)]	Loss: 5.786539
Update Epoch: 0 [5440/20000 (27%)]	Loss: 5.491814
Update Epoch: 0 [5760/20000 (29%)]	Loss: 5.656126
Update Epoch: 0 [6080/20000 (30%)]

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plot loss curve
plt.figure()
plt.plot(range(len(loss_train)), loss_train)
plt.ylabel('train_loss')
plt.savefig('./save/fed_{}_{}_{}_C{}_iid{}.png'.format(args.dataset, args.model, args.epochs, args.frac, args.iid))



In [ ]:
# testing
net_glob.eval()
acc_train, loss_train = test_img(net_glob, dataset_train, args)
acc_test, loss_test = test_img(net_glob, dataset_test, args)
print("Training accuracy: {:.2f}".format(acc_train))
print("Testing accuracy: {:.2f}".format(acc_test))
writer.close()